In [3]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
CUDA_VISIBLE_DEVICES=1
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

import numpy as np
import pandas as pd
import pickle
from tensorflow import keras

from tensorflow.keras.metrics import RootMeanSquaredError as rmse
from tensorflow.keras.models import load_model
from keras import regularizers
from keras import backend
from keras.layers import Dropout, Input, BatchNormalization, concatenate, Activation, MaxPooling2D, AveragePooling2D, Flatten, Masking
from keras.layers import Bidirectional, GRU, Conv2D, TimeDistributed, LSTM, Dense
from keras.models import Model, Sequential

def RMSE(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true)))

def BIAS(y_true, y_pred):
	return backend.mean((y_pred - y_true), axis=-1)


Num GPUs Available:  1
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


MLP Model

In [4]:
def build_model_mlp(trainX):
    model=Sequential()
    model.add(Dense(25,activation='relu',input_shape=(trainX.shape[1],)))
    model.add(Dense(5,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',loss='mse',metrics=[RMSE,BIAS])
    return model


BIGRU Model

In [5]:
def build_model_bigru(trainX):
    model = Sequential()
    model.add(Bidirectional(GRU(32,  input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True)))
    model.add(Dense(units=25, activation='relu'))
    model.add(Dense(units=5, activation='relu'))
    model.add(Dense(units=1))
    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=adam, loss='mse',metrics=[RMSE,BIAS])
    return model

DenseNET Model

In [ ]:
def conv_layer(conv_x, filters):
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Conv2D(filters, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(conv_x)
    conv_x = Dropout(0.2)(conv_x)

    return conv_x


def dense_block(block_x, filters, growth_rate, layers_in_block):
    for i in range(layers_in_block):
        each_layer = conv_layer(block_x, growth_rate)
        block_x = concatenate([block_x, each_layer], axis=-1)
        filters += growth_rate

    return block_x, filters


def transition_block(trans_x, tran_filters):
    trans_x = BatchNormalization()(trans_x)
    trans_x = Activation('relu')(trans_x)
    trans_x = Conv2D(tran_filters, (1, 1), kernel_initializer='he_uniform', padding='same', use_bias=False)(trans_x)
    trans_x = AveragePooling2D((2, 2), strides=(2, 2))(trans_x)

    return trans_x, tran_filters


def dense_net(in_shape,out_shape,filters, growth_rate, dense_block_size, layers_in_block):
    input_img = Input(shape=in_shape)
    x = Conv2D(24, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_img)

    dense_x = BatchNormalization()(x)
    dense_x = Activation('relu')(x)

    dense_x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(dense_x)
    for block in range(dense_block_size - 1):
        dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
        dense_x, filters = transition_block(dense_x, filters)

    dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
    dense_x = BatchNormalization()(dense_x)
    dense_x = Activation('relu')(dense_x)
    dense_x = Flatten()(dense_x)

    output = Dense(out_shape, activation='relu')(dense_x)

    return Model(input_img, output)

dense_block_size = 3
layers_in_block = 3
growth_rate = 4
model_merge = dense_net((26,26,26),1,growth_rate * 2, growth_rate, dense_block_size, layers_in_block)

optimizer = keras.optimizers.RMSprop(lr=0.001)
model_merge.compile(optimizer = optimizer, loss='mse',metrics=[RMSE,BIAS])

CNNGRU Model

In [8]:
def build_model_cnngru(trainX):
    shape = (trainX.shape[1], trainX.shape[2], trainX.shape[3], trainX.shape[4])
    nLstmNeu=32

    model = Sequential()
    # --- CNN
    model.add(TimeDistributed(Conv2D(filters=16,kernel_size=(3,3),padding='same',kernel_regularizer=regularizers.l2(l=0.01),
                                     activation='relu',input_shape=shape)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_regularizer=regularizers.l2(l=0.01),
                                     activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(filters=64,kernel_size=(3,3),padding='same',kernel_regularizer=regularizers.l2(l=0.1),
                                      activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))
    #model.add(TimeDistributed(Dropout(0.5)))

    # --- LSTM
    model.add(Bidirectional(GRU(nLstmNeu, return_sequences=True, activation='tanh')))
    model.add(TimeDistributed(Dense(8, activation='relu',
                                    kernel_initializer="glorot_uniform",kernel_regularizer=regularizers.l2(l=0.01))))
    model.add(TimeDistributed(Dense(1, activation='relu',
                                    kernel_initializer="glorot_uniform",kernel_regularizer=regularizers.l2(l=0.01))))
    # --- Optimizer 
    adam = keras.optimizers.Adam(lr=0.0005)
    model.compile(optimizer=adam, loss='mse',metrics=[RMSE,BIAS])
    
    return model

Load reduced data and existing models

In [6]:
#load data
data = np.load('cnngru_data.npz',allow_pickle=True)
cnngru_X,cnngru_Y = data['arr_0'], data['arr_1']
#load models
model=load_model('cnngru_model.h5',custom_objects={'RMSE':RMSE,'BIAS':BIAS})    #please note that cnngru and densenet models are trained with tensorflow 2.0.0, and load mlp and bigru models are trained with tensorflow 2.7.0 